In [1]:
import os
import pickle
import pandas as pd

In [2]:
axioms = ["TFC1", "TFC2", "MTDC", "LNC1", "LNC2", "TPC", "STMC1", "STMC2", "STMC3", "LB1", "LB2"]
axioms = ["TPC"]
axioms_with_custom_run_files = set(["LNC2", "LB1", "LB2"])
data_home = "/ssd2/arthur/TREC2019/data/"
qrels_path = os.path.join(data_home, 'qrels', "test_qrels")
qrels = {}
for line in open(qrels_path):
    topic_id, _, doc_id, rel = line.split("\t")
    if topic_id in qrels:
        qrels[topic_id].append(doc_id)
    else:
        qrels[topic_id] = [doc_id]
instances_path = os.path.join(data_home, "diagnostics")
paths = {}
assert os.path.isdir(instances_path)
for ax in axioms:
    if not os.path.isfile(instances_path+"/{}-instances".format(ax)):
        print("Instances file for axiom {} is missing".format(ax))
    else:
        paths[ax] = instances_path+"/{}-instances".format(ax)
        
run_files = {
        "QL": os.path.join(data_home, 'runs', "test_distilBert-0.0.run"),
        "BERT": os.path.join(data_home, 'runs', "test_distilBert-1.0.run"),
#         "QL+BERT": os.path.join(data_home, 'runs', "test_distilBert-0.85.run")
}
scores = dict.fromkeys(run_files.keys())

In [3]:
for ax in axioms:
    print(ax)
    instances = pickle.load(open(paths[ax], 'rb'))
    instances_with_relevant = 0
    instances_with_right_ordering = 0
    if len(instances[0]) == 4:
        for topic_id, di, dj, dk in instances:
            topic_id = topic_id.split("-")[0]
            if di in qrels[topic_id] or dj in qrels[topic_id] or dk in qrels[topic_id]:
                instances_with_relevant += 1
                di_score = dj_score = dk_score = 0
                if di in qrels[topic_id]:
                    di_score = 1
                if dj in qrels[topic_id]:
                    dj_score = 1
                if dk in qrels[topic_id]:
                    dk_score = 1
                if dj_score - di_score > dk_score - dj_score:
                    instances_with_right_ordering += 1
        print("\t",len(instances), instances_with_relevant, instances_with_right_ordering, "{:.4f}".format(instances_with_right_ordering/instances_with_relevant))
        continue
    for topic_id, di, dj in instances:
        _topic_id = topic_id.split("-")[0] # This is needed in some cases with custom topic ids. 
        _di = di.split("-")[0] # This is needed for LB2 modified doc_ids
        _dj = dj.split("-")[0] # This is needed for LB2, for instance
        if _di in qrels[_topic_id] or _dj in qrels[_topic_id]:
            instances_with_relevant += 1
            if _di in qrels[_topic_id]:
                instances_with_right_ordering+=1
    print("\t",len(instances), instances_with_relevant, instances_with_right_ordering, "{:.4f}".format(instances_with_right_ordering/instances_with_relevant))


TPC
	 3010246 19666 3572 0.1816


In [4]:
from pytorch_transformers import BertTokenizer
queries_file = "queries/test_queries.tsv"
queries_file = os.path.join(data_home, queries_file)
assert os.path.isfile(queries_file)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
queries = {}
for line in open  (queries_file):
    q_id, query_text = line.strip().split("\t")
    tokenized_query = tokenizer.tokenize(query_text)
    queries[q_id] = tokenized_query

In [6]:
from collections import defaultdict
results = []
relevants = False
# axioms = ["LB1"]
# splits = [("short", first_percentile), ("medium", second_percentile), ("long", last_percentile)]
for name, topic_to_use in [("all", queries.keys())]:
    print(name)
    for ax in axioms:
        instances = pickle.load(open(paths[ax], 'rb'))
        instances = [x for x in instances if x[0] in topic_to_use or ax=="LB1"]
        print("\t",ax, len(instances))
        for method in run_files:
            scores[method] = {}
            if ax in axioms_with_custom_run_files:
                run_file_path = run_files[method].replace("test", ax)
                print(run_file_path)
            else:
                run_file_path = run_files[method]
            if ax == "LNC2":
                run_file_path = run_file_path.replace("LNC2", "LNC2_dev")
            for line in open(run_file_path):
                topic_id, _, doc_id, _, score, _ = line.split()
                pair_id = "{}-{}".format(topic_id, doc_id)
                scores[method][pair_id] = float(score)
            instances_with_relevant = 0
            instances_with_right_ordering = 0
            if len(instances) == 0:
                print("\t\t{}".format(method), 0, 0, "{:.4f}".format(0))
                continue
            if len(instances[0]) == 4:
                for topic_id, di, dj, dk in instances:
                    topic_id = topic_id.split("-")[0]
                    # This is TFC2
                    if (di in qrels[topic_id] or dj in qrels[topic_id] or dk in qrels[topic_id]) or not relevants:
                        instances_with_relevant += 1
                        di_score = scores[method]["{}-{}".format(topic_id, di)]
                        dj_score = scores[method]["{}-{}".format(topic_id, dj)]
                        dk_score = scores[method]["{}-{}".format(topic_id, dk)]
                        if dj_score - di_score > dk_score-dj_score:
                            instances_with_right_ordering += 1
                results.append({"method":method, "dataset": ax+"-"+name, "value": instances_with_right_ordering/instances_with_relevant})
                print("\t\t{}".format(method), instances_with_relevant, instances_with_right_ordering, "{:.4f}".format(instances_with_right_ordering/instances_with_relevant))
                continue
            for topic_id, di, dj in instances:
                _topic_id = topic_id.split("-")[0] # This is needed in some cases with custom topic ids. 
                _di = di.split("-")[0] # This is needed for LB2 modified doc_ids
                _dj = dj.split("-")[0] # This is needed for LB2, for instance
                if (_di in qrels[_topic_id] or _dj in qrels[_topic_id]) or not relevants:
                    instances_with_relevant += 1
                    di_score = scores[method]["{}-{}".format(topic_id, di)]
                    dj_score = scores[method]["{}-{}".format(topic_id, dj)]
                    if ax == "LNC1" or ax=="STMC2" or ax=="LB1":
                        if di_score >= dj_score and (_di in qrels[_topic_id] or not relevants):
                            instances_with_right_ordering += 1
                    elif ax=="STMC3":
                        if di_score <= dj_score and (_di in qrels[_topic_id] or not relevants):
                            instances_with_right_ordering += 1
                    else:
                        if di_score > dj_score and (_di in qrels[_topic_id] or not relevants):
                            instances_with_right_ordering += 1
            results.append({"method":method, "dataset": ax+"-"+name, "value": instances_with_right_ordering/instances_with_relevant})
            print("\t\t{}".format(method), instances_with_relevant, instances_with_right_ordering, "{:.2f}".format(instances_with_right_ordering/instances_with_relevant))
df = pd.DataFrame(results)
df.pivot("dataset", "method",  "value").to_csv("query_stopwords.csv")

all
	 TPC 3010246
		QL 3010246 1325074 0.44
		BERT 3010246 1365750 0.45


In [17]:
all_questions = list(queries.values())

In [158]:
lens = [len(x) for x in all_questions]
relevant_docs = 
# import numpy as np
np.mean(lens)
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))
sw_counter = [sum([1 if x in sw else 0 for x in y]) for y in all_questions ]
# sw_counter = [sum(x) for x in sw_masks]

In [159]:
np.percentile(sw_counter, 100/3), np.percentile(sw_counter, 200/3), np.percentile(sw_counter, 100)

(2.0, 3.0, 12.0)

In [163]:
first_percentile = []
second_percentile = []
last_percentile = []
for q in queries:
    if sum([1 if x in sw else 0 for x in queries[q]]) < np.percentile(sw_counter, 100/3):
        first_percentile.append(q)
    elif sum([1 if x in sw else 0 for x in queries[q]]) < np.percentile(sw_counter, 200/3):
        second_percentile.append(q)
    else:
        last_percentile.append(q)
len(last_percentile), len(first_percentile), len(second_percentile)

(755, 385, 418)

In [12]:
def getcontent(docid, file_name):
    """getcontent(docid, f) will get content for a given docid (a string) from filehandle f.
    The content has four tab-separated strings: docid, url, title, body.
    """
    with open(file_name, encoding='utf-8') as f:
        f.seek(docoffset[docid])
        line = f.readline()
        assert line.startswith(docid + "\t"), f"Looking for {docid}, found {line}"
    return line.rstrip()


In [13]:
lookup_file = "/ssd2/arthur/TREC2019/data/docs/tokenized-msmarco-docs.tsv.offset"
docs_file = "/ssd2/arthur/TREC2019/data/docs/tokenized-msmarco-docs.tsv"
number_of_lines_to_process = 3213835

docoffset = pickle.load(open(lookup_file, 'rb'))

In [22]:
full_run_file = "/ssd2/arthur/TREC2019/data/runs/test_distilBert-{}.run".format(0.0)
viable_topics = set()
for line in open(full_run_file):
    topic_id, _, doc_id, _, _, _ = line.split()
#     if doc_id == qrels[topic_id][0]:
    viable_topics.add(topic_id)

In [23]:
overlaps = dict()

for q in queries:
    if q not in viable_topics:
        continue
    relevant = set(getcontent(qrels[q][0], docs_file).split("\t")[1].split())
    overlap = len(set(queries[q]).intersection(relevant))/len(set(queries[q]))
    overlaps[q] = overlap

In [24]:
import numpy as np
first_percentile = []
second_percentile = []
last_percentile = []
for q in queries:
    if q not in viable_topics:
        continue
    if overlaps[q] < np.percentile(list(overlaps.values()), 100/3):
        first_percentile.append(q)
    elif overlaps[q] < np.percentile(list(overlaps.values()), 200/3):
        second_percentile.append(q)
    else:
        last_percentile.append(q)
print(len(first_percentile), len(second_percentile), len(last_percentile))

456 565 537


In [26]:
np.percentile(list(overlaps.values()), 100/3), np.percentile(list(overlaps.values()), 200/3)

(0.5, 0.75)

In [27]:
import subprocess
trec_eval_path = "/ssd2/arthur/trec_eval/trec_eval"
ndcg_cmd = "{} -q -m ndcg {} {}"
results = []
for alpha in [0.0, 1.0]:
    print(alpha)
    for counter,( name, split) in enumerate([("[0.0, 0.5)", first_percentile), ("[.5, .75)", second_percentile), ("[.75, 1.0]", last_percentile)]):
        print(name)
        full_run_file = "/ssd2/arthur/TREC2019/data/runs/test_distilBert-{}.run".format(alpha)
        out_run_file = "/ssd2/arthur/TREC2019/data/runs/{}_test_distilBert-{}.run".format(counter, alpha)
        with open(out_run_file, "w") as outf:
            for line in open(full_run_file):
                topic_id = line.split()[0]
                if topic_id in split:
                    outf.write(line)
        ndcgs = subprocess.check_output(ndcg_cmd.format(trec_eval_path, qrels_path, out_run_file).split()).decode('utf-8')
        _ndcg = ndcgs.split("\n")[-2].split("\t")[-1]
        print("\talpha: {}\t ndcg: {}".format(alpha, _ndcg))
        if alpha == 0.0: method = "QL" 
        if alpha == 1.0: method = "BERT" 
        if alpha == 0.85: method = "QL+BERT" 
        results.append({"method":method, "dataset": name, "value": _ndcg, "len":len(split)})
df = pd.DataFrame(results)
df.pivot("dataset", "method",  "value").to_csv("term_overlap")

0.0
[0.0, 0.5)
	alpha: 0.0	 ndcg: 0.0176
[.5, .75)
	alpha: 0.0	 ndcg: 0.1627
[.75, 1.0]
	alpha: 0.0	 ndcg: 0.4462
1.0
[0.0, 0.5)
	alpha: 1.0	 ndcg: 0.0419
[.5, .75)
	alpha: 1.0	 ndcg: 0.2739
[.75, 1.0]
	alpha: 1.0	 ndcg: 0.4933


In [242]:
df = pd.DataFrame(results)

In [255]:
df

,method,dataset,value,len
0,QL,"[0.0, 0.5)",0.0448,616
1,QL,"[.5, .75)",0.2095,485
2,QL,"[.75, 1.0]",0.4719,457
3,BERT,"[0.0, 0.5)",0.0905,616
4,BERT,"[.5, .75)",0.3066,485
5,BERT,"[.75, 1.0]",0.5334,457
